<a href="https://colab.research.google.com/github/AugustineKwabenaAntwi/Numerical_Analysis_Project1/blob/main/Project_1_Numerical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Project: Deflation Algorithm
-------------------------------------
Name(s): Augustine Antwi ,Noah Gould, Grace Klunder
Course : Numerical Analysis
Date   : October 2025

Goal:
  Find ALL roots (real or complex) of a polynomial of degree ≤ 5
  using:
    - Newton’s method (primary)
    - Müller’s method (fallback; handles complex roots)
    - Horner’s method (evaluation + synthetic division / deflation)

Polynomial format:
  coeffs = [a_n, a_{n-1}, ..., a_0]   # highest degree first
"""

import math
import cmath


# ============================================================
# (1) Small functions
# ============================================================

def strip_leading_zeros(coeffs):
    """Remove leading zeros at the front, if any."""
    i = 0
    while i < len(coeffs) - 1 and abs(coeffs[i]) == 0:
        i += 1
    return coeffs[i:]


def poly_value_only(coeffs, x):
    """Horner evaluation of P(x) (value only)."""
    p = coeffs[0]
    for i in range(1, len(coeffs)):
        p = p * x + coeffs[i]
    return p


# ============================================================
# (2) Horner evaluation for P(x) and P'(x)
# ============================================================

def horner_eval(coeffs, x):
    """
    Return (P(x), P'(x)) using Horner’s scheme.
    coeffs in descending powers: [a_n, ..., a_0]
    """
    b = coeffs[0]   # accumulator for value
    c = 0           # accumulator for derivative
    for i in range(1, len(coeffs)):
        c = c * x + b  # Build derivative while we're at it
        b = b * x + coeffs[i]  # Standard Horner for the value
    return b, c


# ============================================================
# (3) Newton’s method (primary)
# ============================================================

def newton_root(coeffs, x0, tol=1e-12, max_iter=50, value_tol=1e-10):
    """
    Newton iteration: x_{k+1} = x_k - P(x_k)/P'(x_k).
    Returns (root_estimate, converged_bool).
    """
    x = complex(x0)  # Complex from the start
    for _ in range(max_iter):
        fx, dfx = horner_eval(coeffs, x)
        if abs(fx) < value_tol:  # Close enough to zero?done!
            return x, True
        if abs(dfx) < 1e-14:  #  flat region - can't divide by near-zero
            return x, False
        x_next = x - fx / dfx  # The Newton step
        if abs(x_next - x) < tol:  # Not moving much? Probably converged
            return x_next, True
        x = x_next
    return x, False  # Tried hard but no turn up


# ============================================================
# (4) Müller’s method ( ideal for complex roots)
# ============================================================

def muller_root(coeffs, x0, x1, x2, tol=1e-12, max_iter=80, value_tol=1e-10):
    """
    Müller's method via quadratic interpolation through (x0, x1, x2).
    Returns (root_estimate, converged_bool).
    """
    x0 = complex(x0); x1 = complex(x1); x2 = complex(x2)

    for _ in range(max_iter):
        f0 = poly_value_only(coeffs, x0)
        f1 = poly_value_only(coeffs, x1)
        f2 = poly_value_only(coeffs, x2)

        if abs(f2) < value_tol:  # Already at a root? nice!
            return x2, True

        h0 = x1 - x0
        h1 = x2 - x1
        if h0 == 0 or h1 == 0:  # Points too close together
            return x2, False

        # Fit a parabola through our three points t
        d0 = (f1 - f0) / h0
        d1 = (f2 - f1) / h1
        a = (d1 - d0) / (h1 + h0)
        b = a * h1 + d1
        c = f2

        # Solve quadratic equation :-)
        rad = cmath.sqrt(b*b - 4*a*c)
        denom_plus  = b + rad
        denom_minus = b - rad
        # Pick the denominator that won't give us tiny numbers
        denom = denom_plus if abs(denom_plus) > abs(denom_minus) else denom_minus
        if denom == 0:  # No good denominator? stop!!
            return x2, False

        dx = -2 * c / denom  # Müller's special sauce
        x3 = x2 + dx
        if abs(dx) < tol:  # Converged!
            return x3, True

        # Shift our three points for next iteration
        x0, x1, x2 = x1, x2, x3

    return x2, False  # Out of iterations


# ============================================================
# (5) Synthetic division (deflation) by (x - r)
# ============================================================

def deflate_polynomial(coeffs, root):
    """
    Divide P(x) by (x - root) using Horner synthetic division.
    Returns (quotient_coeffs, remainder).
    """
    n = len(coeffs)
    b = [coeffs[0]]                 # first coefficient drops down
    for i in range(1, n - 1):       # build quotient coefficients
        b.append(coeffs[i] + b[-1] * root)
    remainder = coeffs[-1] + b[-1] * root
    return b, remainder


# ============================================================
# (6) Root polishing (a couple of Newton steps)
# ============================================================

def polish_root(coeffs, r, steps=2):
    """Refine an estimated root with a few Newton steps."""
    x = complex(r)
    for _ in range(steps):
        fx, dfx = horner_eval(coeffs, x)
        if abs(dfx) < 1e-14:  # Can't improve if derivative is tiny
            break
        x = x - fx / dfx
    return x


# ============================================================
# (7)The climax : find ALL roots by deflation
# ============================================================

def find_all_roots(coeffs, tol=1e-12, verbose=True):
    """
    Repeatedly:
      1) try Newton from simple seeds,
      2) if needed, fall back to Müller,
      3) deflate by (x - r),
      4) continue until linear.
    Returns list of roots (complex).
    """
    a = strip_leading_zeros(coeffs[:])
    roots = []

    if len(a) <= 1:  # Constant polynomial or empty? No roots to find!
        return roots

    while len(a) > 2:  # degree ≥ 2
        if verbose:
            print(f"[deflation] current degree = {len(a) - 1}")

        # Step 1: try a few simple Newton seeds
        seeds = [0.0, 1.0, -1.0, 2.0, -2.0]  # These usually work surprisingly well
        root = None
        for s in seeds:
            r, ok = newton_root(a, s, tol=tol)
            if ok:
                root = r
                break

        # Step 2: Müller fallback if Newton didn’t settle
        if root is None:
            r, ok = muller_root(a, 0.0, 0.6 + 0.8j, 0.6 - 0.2j, tol=tol)
            root = r

        # Polish against current polynomial and deflate
        root = polish_root(a, root, steps=2)
        a, rem = deflate_polynomial(a, root)
        if verbose:
            print(f"  remainder after synthetic division = {abs(rem):.2e}")
        roots.append(root)

    # Finish: linear ax + b => root = -b/a
    if len(a) == 2:
        a0, a1 = a
        last_root = -a1 / a0 if abs(a0) > 1e-16 else 0.0
        roots.append(last_root)

    # Final polish against ORIGINAL polynomial; report residuals
    orig = strip_leading_zeros(coeffs[:])
    for i in range(len(roots)):
        roots[i] = polish_root(orig, roots[i], steps=2)

    if verbose:
        print("\nfinal roots and residuals |P(r)|:")
        for r in roots:
            print(f"  r = {r},  |P(r)| = {abs(poly_value_only(orig, r)):.2e}")

    return roots


# ============================================================
# (8) Pretty printer for polynomials
# ============================================================

def print_poly(coeffs):
    """Print P(x) in a tidy algebraic form."""
    coeffs = strip_leading_zeros(coeffs)
    n = len(coeffs) - 1
    terms = []
    for i, a in enumerate(coeffs):
        p = n - i
        if abs(a) < 1e-14:  # Skip tiny coefficients
            continue
        sign = "+" if a >= 0 else "-"
        mag = abs(a)
        if p == 0:
            term = f"{mag:g}"  # Constant term
        elif p == 1:
            term = ("" if math.isclose(mag, 1.0) else f"{mag:g}") + "x"
        else:
            term = ("" if math.isclose(mag, 1.0) else f"{mag:g}") + f"x^{p}"
        terms.append((sign, term))
    if not terms:
        print("P(x) = 0"); return
    sgn, trm = terms[0]
    poly = ("-" if sgn == "-" else "") + trm
    for sgn, trm in terms[1:]:
        poly += f" {sgn} {trm}"
    print("P(x) =", poly)


# ============================================================
# (9) Simple demos / tests (degree ≤ 5)
# ============================================================

if __name__ == "__main__":
    # Test A: distinct real (x-1)(x-2)(x-3)
    print("========== TEST A ==========")
    A = [1, -6, 11, -6]
    print_poly(A)
    rootsA = find_all_roots(A, verbose=True)

    # Test B: purely complex x^2 + 1 = 0 → ±i
    print("\n========== TEST B ==========")
    B = [1, 0, 1]
    print_poly(B)
    rootsB = find_all_roots(B, verbose=True)

    # Test C: repeated root (x-1)^4
    print("\n========== TEST C ==========")
    C = [1, -4, 6, -4, 1]
    print_poly(C)
    rootsC = find_all_roots(C, verbose=True)

    # Test D: mixed (x-2)(x+1)(x^2+1) = x^4 - x^3 - x^2 - 2x + 2
    print("\n========== TEST D ==========")
    D = [1, -1, -1, -2, 2]
    print_poly(D)
    rootsD = find_all_roots(D, verbose=True)

    # Test E: degree-5 all real (x+2)(x+1)(x-1)(x-2)(x-3)
    #         = x^5 - 3x^4 - 4x^3 + 12x^2 + x - 12
    print("\n========== TEST E ==========")
    E = [1, -3, -4, 12, 1, -12]
    print_poly(E)
    rootsE = find_all_roots(E, verbose=True)

    print("\nAll tests complete.")

========== TEST A ==========
P(x) = x^3 - 6x^2 + 11x - 6
[deflation] current degree = 3
  remainder after synthetic division = 0.00e+00
[deflation] current degree = 2
  remainder after synthetic division = 0.00e+00

final roots and residuals |P(r)|:
  r = (1+0j),  |P(r)| = 0.00e+00
  r = (2.0000000000000013+0j),  |P(r)| = 0.00e+00
  r = (3-0j),  |P(r)| = 0.00e+00

========== TEST B ==========
P(x) = x^2 + 1
[deflation] current degree = 2
  remainder after synthetic division = 0.00e+00

final roots and residuals |P(r)|:
  r = -1j,  |P(r)| = 0.00e+00
  r = 1j,  |P(r)| = 0.00e+00

========== TEST C ==========
P(x) = x^4 - 4x^3 + 6x^2 - 4x + 1
[deflation] current degree = 4
  remainder after synthetic division = 3.20e-12
[deflation] current degree = 3
  remainder after synthetic division = 6.66e-16
[deflation] current degree = 2
  remainder after synthetic division = 0.00e+00

final roots and residuals |P(r)|:
  r = (0.9992474720798903+0j),  |P(r)| = 3.20e-13
  r = (1.0007524785872657+0j),